In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd


In [ ]:

# Lê o arquivo parquet
df = pd.read_parquet('data/raw/mimic_iv_ext_onto_sepse_rotulado.parquet')

# Define as colunas de entrada e alvo
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

# Pré-processamento
dados_entrada = df[colunas_entrada].apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)
X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

# Função de avaliação baseada no F1-score médio
def avaliar_caracteristicas(indices, X, y):
    modelo = RandomForestClassifier(n_estimators=50, random_state=42)
    X_sel = X[:, indices]
    scores = cross_val_score(modelo, X_sel, y, cv=5, scoring='f1')
    return scores.mean()

# Algoritmo ACO para seleção de características
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=10,
    n_iteracoes=4,
    alfa=1.5,
    beta=2.0,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=10
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)
    heuristica = mutual_info_classif(X, y)
    heuristica += 1e-6

    melhor_indices = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        print(f"Iteração {iteracao + 1}/{n_iteracoes}")
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        for formiga in range(n_formigas):
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()
            tamanho_conjunto = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )
            indices_sel = np.random.choice(
                range(n_caracteristicas),
                size=tamanho_conjunto,
                replace=False,
                p=probabilidades
            )
            pontuacao = avaliar_caracteristicas(indices_sel, X, y)
            solucoes_iteracao.append(indices_sel)
            pontuacoes_iteracao.append(pontuacao)
            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indices = indices_sel

        feromonio *= (1 - taxa_evaporacao)
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for indice in solucao:
                feromonio[indice] += pontuacao

    return melhor_indices, melhor_pontuacao

# Execução do ACO
melhores_indices, melhor_f1 = selecao_por_colonia_de_formigas(X, y)
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]
print("Melhores características selecionadas:", melhores_nomes)
print("Melhor F1-score médio:", melhor_f1)